In [1]:
import pandas as pd
import numpy as np

import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import string

import arabicstopwords.arabicstopwords as stp

import re

import nltk
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('new.csv')
df2=pd.read_csv('try.csv')

In [50]:
pd.set_option('display.max_columns', 60)

In [3]:
r = [df, df2]
df = pd.concat(r)
df.drop_duplicates(inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 277 entries, 0 to 63
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   links          277 non-null    object
 1   court          277 non-null    object
 2   court_id       277 non-null    object
 3   court_date     277 non-null    object
 4   appeal_court   221 non-null    object
 5   appeal_id      221 non-null    object
 6   appeal_date    169 non-null    object
 7   region         169 non-null    object
 8   city           213 non-null    object
 9   judgment_text  277 non-null    object
 10  appeal_text    169 non-null    object
 11  appeal_data    52 non-null     object
 12  appleal_text   52 non-null     object
dtypes: object(13)
memory usage: 30.3+ KB


In [5]:
df.duplicated().sum()

0

In [6]:
df_Train, df_test = train_test_split(df, test_size=0.2, random_state= 30)

df_train, df_val = train_test_split(df_Train, test_size=0.2, random_state= 30)

In [7]:
print(f'Shape of train:', df_train.shape)
print(f'Shape of validation:', df_val.shape)
print(f'Shape of test:', df_test.shape)

Shape of train: (176, 13)
Shape of validation: (45, 13)
Shape of test: (56, 13)


In [8]:
df_train.isnull().sum()

links              0
court              0
court_id           0
court_date         0
appeal_court      38
appeal_id         38
appeal_date       71
region            71
city              43
judgment_text      0
appeal_text       71
appeal_data      143
appleal_text     143
dtype: int64

In [9]:
df_val.isnull().sum()

links             0
court             0
court_id          0
court_date        0
appeal_court      8
appeal_id         8
appeal_date      17
region           17
city              9
judgment_text     0
appeal_text      17
appeal_data      36
appleal_text     36
dtype: int64

In [10]:
df_train.appeal_court.value_counts(),

(محكمة الاستئناف: المحكمة التجارية    130
 محكمة الاستئناف: محكمة الاستئناف       8
 Name: appeal_court, dtype: int64,)

In [11]:
df_val.appeal_court.value_counts(),

(محكمة الاستئناف: المحكمة التجارية    36
 محكمة الاستئناف: محكمة الاستئناف      1
 Name: appeal_court, dtype: int64,)

In [12]:
df_train['appeal_court'].fillna(df_train['appeal_court'].mode()[0], inplace = True)

In [13]:
df_val['appeal_court'].fillna(df_val['appeal_court'].mode()[0], inplace = True)

In [14]:
df_train.appeal_id.value_counts()

رقم القرار: ٥٥٢٨    2
رقم القرار: ٥٩١     2
رقم القرار: ٧٩٠     2
رقم القرار: ٧٠٤     2
رقم القرار: ٧٠٩     2
                   ..
رقم القرار: ١٢٦     1
رقم القرار: ٥٥٢٥    1
رقم القرار: ٩٦      1
رقم القرار: ٣٤٥٤    1
رقم القرار: ٥١٧     1
Name: appeal_id, Length: 117, dtype: int64

In [15]:
df_val.appeal_id.value_counts()

رقم القرار: ٢٢٢٩    2
رقم القرار: ٣٤٧     2
رقم القرار: ٥٣٨١    1
رقم القرار: ٥٢٥٥    1
رقم القرار: ٥٢٩٠    1
رقم القرار: ٥٤٢     1
رقم القرار: ٢٤٥     1
رقم القرار: ٩٣      1
رقم القرار: ٣٩      1
رقم القرار: ١٦٥٤    1
رقم القرار: ٢٧      1
رقم القرار: ٥٦٩١    1
رقم القرار: ٣٥٩     1
رقم القرار: ١٢٢     1
رقم القرار: ٥٤٩٢    1
رقم القرار: ٦٠٥     1
رقم القرار: ٣٧٩     1
رقم القرار: ٢٤٧     1
رقم القرار: ٥٠٥٠    1
رقم القرار: ٢١١٠    1
رقم القرار: ١٩٩٧    1
رقم القرار: ٥٥٧٠    1
رقم القرار: ٥٤٠٤    1
رقم القرار: ٦٤٦     1
رقم القرار: ٢٠٠٧    1
رقم القرار: ٥٤٩١    1
رقم القرار: ٦٨٨     1
رقم القرار: ١٠٦     1
رقم القرار: ١٩٨٤    1
رقم القرار: ٦٣٩     1
رقم القرار: ٢٤٣٠    1
رقم القرار: ٧٨٥     1
رقم القرار: ٨٤      1
رقم القرار: ٣٢٤٢    1
رقم القرار: ٥١٧     1
Name: appeal_id, dtype: int64

In [16]:
df_train.dropna(subset=['appeal_id'], inplace=True);

In [17]:
df_val.dropna(subset=['appeal_id'], inplace=True);

In [18]:
df_train['appeal_date'] = df_train['appeal_date'].str.replace('تاريخه:','')

In [19]:
df_val['appeal_date'] = df_val['appeal_date'].str.replace('تاريخه:','')

In [20]:
df_train['appeal_id'] = df_train['appeal_id'].str.replace('رقم القرار:','')

In [21]:
df_val['appeal_id'] = df_val['appeal_id'].str.replace('رقم القرار:','')

In [22]:
df_val['appeal_date'] = df_val['appeal_id'].str.replace('رقم القرار:','')

In [23]:
df_train['court'] = df_train['court'].str.replace('المحكمة:','')

In [24]:
df_train['court_id'] = df_train['court_id'].str.replace('رقم القضية - القرار:','')

In [25]:
df_val['court_id'] = df_val['court_id'].str.replace('رقم القضية - القرار:','')

In [26]:
df_train['court_date'] = df_train['court_date'].str.replace('تاريخها: ','')

In [27]:
df_val['court_date'] = df_val['court_date'].str.replace('تاريخها: ','')

In [28]:
df_train['appeal_court'] = df_train['appeal_court'].str.replace('محكمة الاستئناف: ','')

In [29]:
df_val['appeal_court'] = df_val['appeal_court'].str.replace('محكمة الاستئناف: ','')

In [30]:
df_train['region'] = df_train['region'].str.replace('المدينة: ','')

In [31]:
df_val['region'] = df_val['region'].str.replace('المدينة: ','')

In [32]:
df_train['city'] = df_train['city'].str.replace('المدينه: ','')

In [33]:
df_val['city'] = df_val['city'].str.replace('المدينه: ','')

In [34]:
df_train.city.value_counts()

الرياض             85
المدينة المنورة     8
الدمام              7
بريدة               5
Name: city, dtype: int64

In [35]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
df_train.judgment_text = df_train.judgment_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))
# df_train.appeal_text = df_train.appeal_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))

In [53]:
df_train.judgment_text = df_train.judgment_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))
df_val.judgment_text = df_val.judgment_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))
# df_val.appeal_text = df_val.appeal_text.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))

In [37]:
df_train

,links,court,court_id,court_date,appeal_court,appeal_id,appeal_date,region,city,judgment_text,appeal_text,appeal_data,appleal_text
119,القضية رقم ٢٢٩ لعام ١٤٤١ هـ,المحكمة التجارية,٢٢٩,١٤/١/١٤٤٢,المحكمة التجارية,٩٦٧,٩/٢/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
124,القضية رقم ٦٠٩٨ لعام ١٤٤١ هـ,المحكمة التجارية,٦٠٩٨,١٣/١٠/١٤٤٢,المحكمة التجارية,٢٠٣,٨/٢/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
81,القضية رقم ٤٠٣١ لعام ١٤٤٢ هـ,المحكمة التجارية,٤٠٣١,٢٠/١٠/١٤٤٢,المحكمة التجارية,٥٤٦٧,١٢/٢/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
62,القضية رقم ١٧٧٤ لعام ١٤٤٢ هـ,المحكمة التجارية,١٧٧٤,٢٢/٨/١٤٤٢,المحكمة التجارية,٤٥٦٧,٥/٣/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
121,القضية رقم ٧٣٠٩ لعام ١٤٤٢ هـ,المحكمة التجارية,٧٣٠٩,٢٣/١٢/١٤٤٢,المحكمة التجارية,٣٨٩,٨/٢/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,القضية رقم ١١١٠ لعام ١٤٤٢ هـ,المحكمة التجارية,١١١٠,١٤/٩/١٤٤٢,المحكمة التجارية,٥١٥٩,٨/٢/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
46,القضية رقم ٣١٤٥ لعام ١٤٤١ هـ,المحكمة التجارية,٣١٤٥,٧/٧/١٤٤١,المحكمة التجارية,٢٦٣٠,NaN,NaN,NaN,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,NaN,تاريخه: ٥/٣/١٤٤٣,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...
102,القضية رقم ٢٠٧٧ لعام ١٤٤٢ هـ,المحكمة التجارية,٢٠٧٧,١٦/٨/١٤٤٢,المحكمة التجارية,٤٧١٦,٨/٢/١٤٤٣,منطقة الرياض,الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN
43,القضية رقم ٣٣٢ لعام ١٤٤٢ هـ,المحكمة العامة,٣٣٢,٢٦/١٠/١٤٤٢,المحكمة التجارية,٥٥٢٨,٢٤/١٠/١٤٤٣,منطقة الرياض,بريدة,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد\n...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,NaN,NaN


# Preprocessing

### Remove stopwords

In [38]:
df_train['judgment_text'] = df_train['judgment_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stp.stopwords_list()]))
# df_train['appeal_text'] = df_train['appeal_text'].apply(lambda y: ' '.join([word for word in y.split() if word not in stp.stopwords_list()]))


### Remove digits

In [39]:
df_train['judgment_text'] = df_train['judgment_text'].apply(lambda x: re.sub('\w*\d\w*', '', x))
# df_train['appeal_text'] = df_train['appeal_text'].apply(lambda x: re.sub('\w*\d\w*', '', x))

In [40]:
df_train['judgment_text'] = df_train['judgment_text'].str.replace('\n','')

In [41]:
df_train['judgment_text'] = df_train['judgment_text'].apply(lambda x: ISRIStemmer().suf32(x))


In [42]:
ISRIStemmer().suf32("منجزين")

'منجز'

In [43]:
df_train['judgment_text'][0]

'الحمد لله والصلاة والسلام رسول ﷲ الدائرة التجارية الثانية والعشرون وبناءً القضية رقم  لعام  المقامة عبدﷲ ابراهيم المحمود سجل تجاري ضد تغريد بنت محمد صفير هوية وطنية القاضي عمر مبارك الدوسري رئيسا الوقائع تتلخص وقائع الدعوى وبالقدر اللازم لإصدار الحكم وردت المحكمة التجارية لائحة دعوى مقدمة المدعي وكالة ضد المدعى والمثبتة بياناتها ملف القضية جاء لائحة الدعوى اتفق موكلي الجهة المدعى بتأريخ   لتنفيذ أعمال الموقع الخارجي لمشروع القوات البحرية الملكية السعودية الجبيل وتمثل  أعمال المشروع والمتمثلة أعمال السفلتة وغرف التفتيش وعددها  غرفة ومدة العقد  خمسة عشر يوماً تنجز الجهة المدعى  الأعمال وعدد  غرف تفتيش عدد  غرفة واستلموا كامل المبلغ وقدره  مئتان واثنان وعشرون الف ريال إتمام المشروع والمستحق الفعلي  مئة وعشرة ألف ريالاً أطلب يلزم الجهة المدعى مؤسسة قمة الصحراء للمقاولات العامة بدفع لموكلتي مبلغ وقدره  مئة واثنا عشر ألف ريالاً قيدت الأوراق قضية بالرقم المشار بعالية أحيلت الدائرة فباشرت نظرها النحو الموضح بمحاضر الضبط جلسة يوم  حضر وكيل المدعي بوكالة رقم  ذكر المدعي دعواه مقامة شركة بيت الج

# remoive english words

In [54]:
df_train.judgment_text = df_train.judgment_text.apply(lambda x: re.sub('([@A-Za-z0-9_]+)|[^\w\s]|#|http\S+', '', x) )

In [47]:
re.sub('([@A-Za-z0-9_]+)|[^\w\s]|#|http\S+', '', text)

# TF-IDF

In [55]:
cv_tfidf = TfidfVectorizer()
x = cv_tfidf.fit_transform(df_train['judgment_text']).toarray()

In [56]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [57]:
dt_tfidf = pd.DataFrame(x,columns=cv_tfidf.get_feature_names())
dt_tfidf

,آب,آت,آثار,آثارها,آجل,آخر,آخرين,آداب,آر,آكد,آل,آلاء,آلاف,آلة,آلت,آله,آلي,آلية,آمنوا,آن,آند,آنف,آنفا,آنفة,آولا,أأن,أبدا,أبدت,أبدوا,أبدى,...,يوجهون,يود,يودان,يودع,يودون,يورد,يورو,يوسف,يوصف,يوصل,يوضح,يوفر,يوفق,يوفقكم,يوفي,يوقع,يوكل,يوليو,يوم,يوما,يومنا,يومي,يوميا,يومية,يومين,يونيه,يونيو,يوهمني,يي,ييثبت
0,0.0,0.0,0.0,0.0,0.0,0.025249,0.10141,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.020206,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025562,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.011878,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.009506,0.0,0.000000,0.019674,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024050,0.012836,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.031663,0.0,0.0,0.0,0.0,0.010023,0.0,0.044987,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022493,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.024358,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.032891,0.0,0.0,0.0,0.0,0.000000,0.0,0.057486,0.0,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.09231,0.0
134,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0
135,0.0,0.0,0.0,0.0,0.0,0.015993,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.012798,0.0,0.000000,0.026489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.057446,0.0,0.000000,0.0,0.00000,0.0
136,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0
